In [ ]:
# Write behavioral videos

# J. Carpenter, 2020.

In [129]:
# import libraries
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Load dataset
fileLoc = "D:\\Data\\Dataset\\24116.mat"
matfile = sio.loadmat(fileLoc) # load .mat file

C:\Users\17145\Anaconda3\lib\site-packages\scipy\io\matlab\mio.py:218: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [289]:
# define functions
# convert to OOP

def getUnitData(matfile, sessNum, unitNum):
    
    """
    The following data will be loaded (via scipy.io):
    
        'SpikeTrain':    Sx1 array, where S is the number of sessions for the animal. 
                         Each session (S) contains a Nx1 vector, where N is the number of 
                         neurons (units) recorded in a given session.
                      
        'pos':           Sx1 array containing information about time and the positions of 2 LEDs.
        
        'UniqueID':      Sx1 array, where each array contains an Nx1 vector. Each element is a 
                         string value, containing the unique cell ID for the given unit.
        
        'trialType':    Sx1 array, containing a string value of session type (FM or OF, for
                        'Foster Maze' or 'Open Field', respectively. 'TS' might also be present,
                         which stands for 'Training Session'.
                         
    Outputs:
    
        'time':         tx1 array, where t is the number of timestamps for the session.
    """
    
    # pull out relevant arrays
    SpikeTrain = matfile["SpikeTrain"][0]
    pos = matfile["pos"][0] # time & LEDs
    UniqueID = matfile["UniqueID"][0]
    trialType = matfile["trialType"][0]
    
    # parse position vector
    time = pos[sessNum][:,0]
    led1A = pos[sessNum][:,1]
    led1B = pos[sessNum][:,2]
    led2A = pos[sessNum][:,3]
    led2B = pos[sessNum][:,4]
    
    # trial type, spike train, unique ID #
    trialNow = trialType[sessNum][0]
    spikesNow = SpikeTrain[sessNum][0][unitNum][0]
    idNow = UniqueID[sessNum][0][unitNum][0][0]
    
    # get window maximums and minumums
    minX = matfile["sessInfo"][0][sessNum][0]["window_min_x"][0][0][0][0]
    maxX = matfile["sessInfo"][0][sessNum][0]["window_max_x"][0][0][0][0]
    minY = matfile["sessInfo"][0][sessNum][0]["window_min_y"][0][0][0][0]
    maxY = matfile["sessInfo"][0][sessNum][0]["window_max_y"][0][0][0][0]
    
    return

######################################################################
    
    
def plotBehavior(minX, minY, maxX, maxY, timeNow, led1Now, led2Now):
    
    """
    plots behavior for a single timepoint.
    """
    
    fig, ax = plt.subplots()
    ax.hlines(y = minY, xmin = minX, xmax = maxX) #minY
    ax.hlines(y = maxY, xmin = minX, xmax = maxX) # maxY
    ax.vlines(x = minX, ymin = minY, ymax = maxY) # minX
    ax.vlines(x = maxX, ymin = minY, ymax = maxY) # maxX
    ax.plot(led1Now, led2Now) # plot animal position
    ax.set_xlabel("x");
    ax.set_ylabel("y");
    ax.set_title("title");
    
    plt.show()
    return

In [290]:
getUnitData(matfile, 1, 1)
len(time)

111044

In [ ]:
# pull out data for current session > neuron

for sessNum in range(1,len(pos)+1): # through sessions
    for unitNum in range(1,len(SpikeTrain[sessNum][0])): #through units
        getUnitData(matfile, sessNum, unitNum) 
        for t in range(1,len(time)+1): # through timestamps
            plotBehavior(minX, minY, maxX, maxY, timeNow, led1Now, led2Now)

In [225]:
# grab mock data
sess = 10
minX = float(matfile["sessInfo"][0][sess][0]["window_min_x"][0][0][0][0])
maxX = float(matfile["sessInfo"][0][sess][0]["window_max_x"][0][0][0][0])
minY = float(matfile["sessInfo"][0][sess][0]["window_min_y"][0][0][0][0])
maxY = float(matfile["sessInfo"][0][sess][0]["window_max_y"][0][0][0][0])